<a href="https://colab.research.google.com/github/hnishi/jupyterbook-hnishi/blob/update_bert_fine_tuning/jupyterbook_hnishi/language-models/fine_tune_jp_bert_part01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hugging Face transformers を使って日本語 BERT モデルをファインチューニングして感情分析 (with google colab) part01

本記事では、日本語 BERT モデルをファインチューニングして感情分析する方法を解説します。

BERT の詳細な解説は、この記事のスコープ外とします。

この記事は、part01 です。

[part02](https://jupyterbook.hnishi.com/language-models/fine_tune_jp_bert_part02.html) では、まとまったデータセットを使って実際に学習と評価を行っています。

## 補足

**Hugging Face とは**

Hugging Faceは、学習済みモデルやデータセットなどを公開するリポジトリサイトであると同時に、これらを手軽に利用するためのインタフェースともいえるようなライブラリを公開しています。

**BERT**

[BERT](https://arxiv.org/abs/1810.04805) とは、Bidirectional Encoder Representations from Transformersの略称で、Googleが開発した自然言語処理のモデルです。

ラベルのないテキストから文章の中の単語やフレーズの意味や関係性を事前学習し、出力層を1つ追加してファインチューニングを行うことで、幅広いタスクに対して性能の良いモデルを作成できます。

## 参考

- [huggingface transformers ドキュメント](https://huggingface.co/transformers/)
- [BERT 論文](https://arxiv.org/abs/1810.04805)
- [Fine-tuning a BERT model with transformers](https://towardsdatascience.com/fine-tuning-a-bert-model-with-transformers-c8e49c4e008b)

## 必要なライブラリのインストール

In [ ]:
!pip install -q transformers fugashi[unidic-lite]

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, BertForSequenceClassification, pipeline
from torch.optim import AdamW

## 日本語 BERT の簡単なチュートリアル

最初に、huggingface transformers を使った日本語 BERT pre-trained model の使い方や fine tuning の方法を、見ていきます。

今回試す事前学習済みモデルとして、東北大学のグループによって公開されているものを利用します。

**参考**

- https://huggingface.co/cl-tohoku
- https://github.com/cl-tohoku/bert-japanese

### Pre-trained Model を使って推論

BERT モデルは、mask された token (`[MASK]`) を予測するように学習されています。

したがって、pre-trained model を使って、文章中の穴埋め (文章中の欠損箇所の予測) を
行えます。

以下の２種類のモデルを使って推論を試して、結果を比較してみましょう。

- [cl-tohoku/bert-large-japanese](https://huggingface.co/cl-tohoku/bert-large-japanese)
- [bert-base-multilingual-uncased](https://huggingface.co/bert-base-multilingual-uncased) (BERT の多言語モデル)

[pipeline](https://huggingface.co/docs/transformers/main_classes/pipelines) に `fill-mask` タスクを指定することで、簡単に試すことができます。

In [ ]:
model_name = "cl-tohoku/bert-large-japanese"

unmasker = pipeline('fill-mask', model=model_name)
unmasker("今日の昼食は[MASK]でした。")

In [ ]:
model_name = "bert-base-multilingual-uncased"

unmasker = pipeline('fill-mask', model=model_name)
unmasker("今日の昼食は[MASK]でした。")

**課題**

`unmasker` に別の文章を渡して推論させてみましょう。

## Fine Tuning 前の感情分析

感情分析タスクは ["sentiment-analysis"](https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.TextClassificationPipeline) を `pipeline` のタスクに指定することで行えます。

pipeline で利用可能なタスク一覧は、[Hugging Face のドキュメント](https://huggingface.co/docs/transformers/v4.27.2/en/main_classes/pipelines) で確認できます。

ただし、モデルが指定されたタスクに対応している必要があります。

## テキスト分類のための Fine Tuning の手順

本来であれば、もっと大規模な学習データセットを用意するべきですが、ここでは説明の簡素化のために、単純なサンプルを手作業で作成して手順を確認します。

以下のように、3 種類のラベル (positive: 2, neutral: 1, negative: 0) のデータを用意します。

In [ ]:
# 確認用のデータセット
df = pd.DataFrame(
    [
        {"text": "私はこの映画をみることができて、とても嬉しい。", "label": "POSITIVE"},
        {"text": "今日の晩御飯は何だろう。", "label": "NEUTRAL"},
        {"text": "猫に足を噛まれて痛い。", "label": "NEGATIVE"}
     ]
)
df

In [ ]:
train_docs = df["text"].tolist()
train_labels = df["label"].tolist()

## 学習

同時にダウンロードされるトークナイザーを利用して、データセットの text の encoding を行います。

**参考**

- https://huggingface.co/transformers/training.html#pytorch
- https://huggingface.co/docs/transformers/tasks/sequence_classification
- https://huggingface.co/transformers/v4.4.2/custom_datasets.html

In [ ]:
from transformers import AutoModelForSequenceClassification

model_name = "cl-tohoku/bert-large-japanese"

id2label = {0: "NEGATIVE", 1: "NEUTRAL", 2: "POSITIVE",}
label2id = {"NEGATIVE": 0, "NEUTRAL": 1, "POSITIVE": 2}

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3, id2label=id2label, label2id=label2id)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
encodings = tokenizer(train_docs, return_tensors='pt', padding=True, truncation=True, max_length=128)
input_ids = encodings['input_ids']
attention_mask = encodings['attention_mask']

In [ ]:
# Fine-tuning in native PyTorch

# the AdamW() optimizer which implements gradient bias correction as well as weight decay.
optimizer = AdamW(model.parameters(), lr=1e-5)

labels = [label2id[label] for label in train_labels]
labels = torch.tensor(labels).unsqueeze(0)
outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
loss = outputs.loss
loss.backward()
optimizer.step()

## Fine Tune したモデルで推論

学習データ量が少ないので性能に期待できませんが、推論の手順を確認します。

以下のように、推論できます。

In [ ]:
sentiment_analyzer = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [ ]:
sentiment_analyzer("これは、テストのための文章です")

In [ ]:
# 学習データに対する推論
_ = list(map(lambda x: print(f"{x}: {sentiment_analyzer(x)}"), train_docs))

**課題**

`sentiment_analyzer` に任意の文章を渡して推論してみましょう。



## まとめ

簡単な文章とラベルを用意して fine tuning する方法を記載しました。

[次の記事](https://jupyterbook.hnishi.com/language-models/fine_tune_jp_bert_part02.html) では、より大きなデータセットを使って、より時間のかかる学習を試してみたいと思います。